In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain

In [ ]:
df_paths = pd.read_csv('path.csv', sep=';')
df_paths.head()

In [ ]:
# TODO - handle matchs failures
s_chosen_modes = df_paths.SERVICES.dropna().str.split(' ').apply(lambda x: [mode for mode in x if mode != 'WALK']).apply(set).apply(list)

display(s_chosen_modes.apply(len).describe())

s_vp = s_chosen_modes.apply(lambda x: x[0] == 'PersonalVehicle' if len(x) > 0 else False)
display(s_vp.value_counts(normalize=True))

plt.figure()
s_modal_share = s_chosen_modes.value_counts()
s_modal_share.plot.bar()
plt.show()

ls_cols = []
for mode in set(chain.from_iterable(s_chosen_modes)):
    ls_cols.append(s_modal_share.index.map(lambda x: mode in x).astype(int)*s_modal_share)
df_mode_share = pd.concat(ls_cols, axis=1, keys=set(chain.from_iterable(s_chosen_modes)))
df_mode_share.sum().plot.bar()
plt.show()
ls_modes = list(set(chain.from_iterable(s_chosen_modes)))
for mode in ls_modes:
    df_paths[mode]=df_paths.SERVICES.dropna().str.contains(mode).astype(int)

df_paths['TIME_10min'] = pd.to_datetime(df_paths['TIME']).dt.floor("10min").dt.time
df_paths.groupby('TIME_10min')[ls_modes].sum().plot()
plt.show()
df_paths['TC_MS'] = df_paths['BUS_MS']+df_paths['TRAM_MS']+ df_paths['METRO_MS']>0
df_dynamic_modalsplit = df_paths.groupby('TIME_10min')['TC_MS', 'PersonalVehicle'].sum()#.plot()
df_dynamic_modalsplit.divide(df_dynamic_modalsplit.sum(axis=1), axis=0).plot()
plt.show()